In [1]:
from dataAnalyzer import *
from private_tool import *
from dataHandler import *
from numpy import array

In [2]:
ExperimentsNum = 3
Experiments = [MotorImageryDataset(parentDirectory(dir=currDir, separator="\\", n=1)+'/bcicompetitionIV2a/A0'+str(i+1)+'T.npz') for i in range(ExperimentsNum)]

In [3]:
channels = [7, 9, 10, 11]
samplingRate = 250 #Hz
trialsByClasses = array([i.get_trials_by_classes(channels=channels) for i in Experiments])

In [4]:
trialsByClasses.shape

(3,)

In [5]:
channelNum = len(channels)
m = len(trialsByClasses[0]["left"])
channelNum, m

(4, 4)

In [6]:
bandpassedTrialsByClasses = [{i:[[] for _ in range(channelNum)] for i in j.mi_types.values()} for j in Experiments]
for k in range(ExperimentsNum):
    for key in trialsByClasses[k].keys():
        for i in range(channelNum):
            bandpassedTrialsByClasses[k][key][i] = np.array([bandpass_filter(data=trialsByClasses[k][key][i][j], 
                                                                sample_rate=samplingRate, 
                                                                cutoff_low=5, 
                                                                cutoff_high=50)\
                                                for j in range(len(trialsByClasses[k][key][i]))])
    

In [7]:
bandpassedTrialsByClasses[2]["left"][2].shape

(69, 1875)

In [21]:
leftTrials = array([[bandpassedTrialsByClasses[i]["left"][j] for j in range(channelNum)] for i in range(ExperimentsNum)])
leftTrials = array([array([array([array(leftTrials[i][k][j]) for k in range(channelNum)]) for j in range(len(leftTrials[i][0]))]) for i in range(ExperimentsNum)])
# i: experiment
# j: trial

# 현재 상태는 experiment - channel - trial - signal
# experiment - trial - channel - signal로 수정하자

minTrialsNum = min([len(leftTrials[i]) for i in range(len(Experiments))])
leftTrials = array([[i[j] for j in range(minTrialsNum)] for i in leftTrials])

C:\Users\jihun\AppData\Local\Temp\ipykernel_15064\3857896541.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  leftTrials = array([[bandpassedTrialsByClasses[i]["left"][j] for j in range(channelNum)] for i in range(ExperimentsNum)])
C:\Users\jihun\AppData\Local\Temp\ipykernel_15064\3857896541.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  leftTrials = array([array([array([array(leftTrials[i][k][j]) for k in range(channelNum)]) for j in range(len(leftTrials[i][0]))]) for i in range(ExperimentsNum)])


In [23]:
leftTrials[1].shape
# leftTrials = [[i[j] for j in range(minTrialsNum)] for i in leftTrials]

(67, 4, 1875)

In [25]:
left_csp_filter = CSP_filter(len(leftTrials[1]), leftTrials[0])